In [ ]:
#Elixhauser: https://github.com/MIT-LCP/mimic-code/blob/master/concepts/other-languages/4-elixhauser.sql

In [ ]:
import os, re
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [ ]:
icds=pd.read_csv('data-raw/mimic/DIAGNOSES_ICD.csv')
drgs=pd.read_csv('data-raw/mimic/DRGCODES.csv')
d_icd=pd.read_csv('data-raw/mimic/D_ICD_DIAGNOSES.csv')

In [ ]:
icds.head(5)

In [ ]:
def convert_icd9(x): 
    icd9='unknown'
    if str(x)=='nan': icd9=='unknown'
    elif x[0] in ['V', 'E']: icd9= x[0:-2]+'.'+x[-2::]
    else: icd9=x[:-2]+'.'+x[-2::]
    if icd9.isnumeric():
        if int(icd9)<1000: return icd9
        elif int(icd9)<1000: return icd9/10
    else: return icd9

In [ ]:
icds['icd9_converted']=icds.ICD9_CODE.map(lambda x: (convert_icd9(x)))

In [ ]:
def icd9_elix(x): 
    if (x[0].isalpha()): 
        if (x == 'V45'): return 'cardiac_arrhythmias'
        elif (x=='V42.2'): return 'valvular_disease'
        elif (x=='V43.4'): return 'peripheral_vascular'
        elif (x=='V42.7'): return 'liver_disease'
        elif (x == 'V42')| (x == 'V45.1')| (x == 'V56')| (x == 'V56.8'): return 'renal_failure'
        elif (x == 'V12.71'): return 'peptic_ulcer'
        elif (x=='V10.71') | (x=='V10.72') | (x=='V10.79'): return 'lymphoma'
        elif ('V10' in x): return 'solid_tumor'
        elif (x=='V113'): return 'alcohol_abuse'
        else: return 'unknown_other'
    elif (x[0].isnumeric()): 
        if  (x == '398.91') | (x == '402.11') | (x == '402.91') | (x == '404.11') | (x == '404.13') | (x == '404.91') | (x == '404.93') | ((float(x) >= 428) & (float(x)<=428.9)): 
            return 'congestive_heart_failure'
        elif (x == '426.1')| (x == '426.11')| (x == '426.13')| (((float(x) >= 426.2) & (float(x)<=426.53)))| (((float(x) >=426.6) & (float(x)<=426.89)))| (x == '427')| (x == '427.2')| (x == '427.31')| (x == '427.6')| (x == '427.9')| (x == '785')| (x == '53.3'): 
            return 'cardiac_arrhythmias'
        elif ((float(x) >= 93.2) & (float(x)<= 93.24))| ((float(x)>= 394) &  (float(x)<=397.1))| ((float(x)>= 424) & (float(x)<=424.91))| ((float(x) >= 746.3) & (float(x)<=746.6))| (x == '43.3'): 
            return 'valvular_disease'
        elif ((float(x)>=416) & (float(x)<416.9) | (x=='417.9')): 
            return 'pulmonary_circulation'
        elif ((float(x) >= 440) & (float(x)<=440.9))| (x == '441.2')| (x == '441.4')| (x == '441.7')| (x == '441.9')| ((float(x) >= 443.1) & (float(x)<=443.9))| (x == '447.1')| (x == '557.1')| (x == '557.9'): 
            return 'peripheral_vascular'
        elif (x == '401.1')| (x == '401.9')| (x == '402.1')| (x == '402.9')| (x == '404.1')| (x == '404.9')| (x ==  '405.11')| (x == '405.19')| (x ==  '405.91')| (x == '405.99'): 
            return 'hypertension'
        elif ((float(x) >= 342) & (float(x)<=342.12)) | ((float(x)>=342.9) & (float(x)<=344.9)): 
            return 'paralysis'
        elif (x == '331.9')| (x == '332')| (x == '333.4')| (x == '333.5')| ((float(x) >= 334) & (float(x)<= 335.9))| (x == '340')| ((float(x) >= 341.1) & (float(x)<= 341.9))| ((float(x) >= 345) & (float(x)<=345.11))| ((float(x) >= 345.4) & (float(x)<= 345.51))| ((float(x) >= 345.8) & (float(x)<=345.91))| (x == '348.1')| (x == '348.3')| (x == '780.3')| (x == '784.3'):
            return 'other_neurological'
        elif ((float(x) >= 490) & (float(x)<=492.8))| ((float(x) >= 493) & (float(x)<=493.91))| (x == '494')| ((float(x) >= 495) & (float(x)<=505))| (x == '506.4'): 
            return 'chronic_pulmonary'
        elif ((float(x)>=250) & (float(x)<=250.33)): 
              return 'diabetes_uncomplicated'
        elif ((float(x)>=250.4) & (float(x)<=250.73)) | ((float(x)>=250.9) & (float(x)<=250.93)): 
            return 'diabetes_complicated'
        elif ((float(x)>=243) & (float(x)<=244.2)) | (x=='244.8') | (x=='244.9'): 
            return 'hypothyroidism'
        elif   (x == '403.11 ')| (x == '403.91')| (x == '404.12')| (x == '404.92')| (x == '585')| (x == '586))'): 
            return 'renal_failure'
        elif ((float(x)>=571.4) & (float(x)<=571.49)) | (x == '70.32')| (x == '70.33')| (x == '70.54')| (x == '456')| (x == '456.1')| (x == '456.2')| (x == '456.21')| (x == '571')| (x == '571.2')| (x == '571.3')| (x == '571.5')| (x == '571.6')| (x == '571.8')| (x == '571.9')| (x == '572.3')| (x == '572.8'): 
            return 'liver_disease'
        elif (x == '531.7')| (x == '531.9')| (x == '532.7')| (x == '532.9')| (x == '533.7')| (x == '533.9')| (x == '534.7')| (x == '534.9'):
            return 'peptic_ulcer'
        elif ((float(x)>=42) & (float(x)<=44.9)): 
            return 'aids'
        elif ((float(x)>=200) & (float(x)<=202.38)) | ((float(x)>=202.5) & (float(x)<=203.01)) | ((float(x)>=203.8) & (float(x)<=203.81)) | (x=='238.6') | (x=='273.3'): 
            return 'lymphoma'
        elif ((float(x)>=196) & (float(x)<=199.1)): 
            return 'metastatic_cancer'
        elif ((float(x)>=140) & (float(x)<=172.9)) | ((float(x)>=174) & (float(x)<=175.9)) | ((float(x)>=179) & (float(x)<=195.8)): 
            return 'solid_tumor'
        elif ((float(x)>=710) & (float(x)<=710.9)) | ((float(x)>=714) & (float(x)<=714.9)) | ((float(x)>=720) & (float(x)<=720.9)) | (x=='701') | (x=='725'): 
            return 'rheumatoid_arthritis'
        elif ((float(x)>=2860) & (float(x)<=2869)) | ((float(x)>=287.3) & (float(x)<=287.5)) | (x=='287.1'): 
            return 'coagulopathy'
        elif (x=='278'): 
            return 'obesity'
        elif ((float(x)>=260) & (float(x)<=263.9)): 
            return 'weight_loss'
        elif ((float(x)>=276) & (float(x)<=276.9)): 
            return 'fluid_electrolyte'
        elif (x=='2800'): 
            return 'blood_loss_anemia'
        elif ((float(x)>=280.1) & (float(x)<=281.9)) | (x=='285.9'): 
            return 'deficiency_anemias'
        elif ((float(x)>=303.9) & (float(x)<=303.93)) | ((float(x)>=305) & (float(x)<=305.03)) | (x=='291.1') | (x=='291.2') | (x=='291.5') | (x=='291.8') | (x=='291.9'): 
            return 'alcohol_abuse'
        elif ((float(x)>=305.2) & (float(x)<=305.93)) | ((float(x)>=304) & (float(x)<=304.93)) |((float(x)>=292.82) & (float(x)<=292.89)) | (x=='292') | (x=='292.9') : 
            return 'drug_abuse'
        elif ((float(x)>=295) & (float(x)<=298.9)) | ((float(x)>=299.1) & (float(x)<=299.11)): 
            return 'psychoses'
        elif (x == '300.4')| (x == '301.12')| (x == '309')| (x == '309.1')| (x == '311'): 
            return 'depression'
        else: return 'unknown_other'
    else: return 'unknown_other'

In [ ]:
icds['elixhauser_category']=icds.icd9_converted.map(lambda x: icd9_elix(str(x)))

In [ ]:
icds.elixhauser_category.value_counts()

In [ ]:
drg_final=drgs[drgs.DRG_TYPE=='HCFA']

In [ ]:
def convert_drg(x): 
    if ((x >= 103) & (x <= 108))| ((x >= 110) & (x <= 112))| ((x >= 115) & (x <= 118))| ((x >= 120) & (x <= 127))| (x == 129)| ((x >= 132) & (x <= 133))| ((x >= 135) & (x <= 143)): 
        return 'cardiac'
    elif ((x >= 302) & (x <= 305)) | ((x >= 315) & (x <= 333)): return 'renal'
    elif ((x >= 199) & (x <= 202))| ((x >= 205) & (x <= 208)): return 'liver'
    elif ((x >= 400) & (x <= 414))| (x == 473)| (x == 492)| (x == 10)| (x == 11)| (x == 64)| (x == 82)| (x == 172)| (x == 173)| (x == 199)| (x == 203)| (x == 239)| ((x >= 257) & (x <= 260))| (x == 274)| (x == 275)| (x == 303)| (x == 318)| (x == 319)| (x == 338)| (x == 344)| (x == 346)| (x == 347)| (x == 354)| (x == 355)| (x == 357)| (x == 363)| (x == 366)| (x == 367)| ((x >= 406) & (x <= 414)): 
                                      return 'cancer'
    elif (x==88): return 'copd'
    elif ((x>=130) & (x<=131)): return 'peripheral_vascular'
    elif (x==134): return 'hypertension'
    elif ((x>=14) & (x<=17)) | (x==5): return 'cerebrovascular'
    elif ((x>=1) & (x<=35)): return 'nervous_system'
    elif ((x>=96) & (x<=98)): return 'asthma'
    elif ((x>=294) & (x<=295)): return 'diabetes'
    elif x==250: return 'thyroid'
    elif ((x>=300) & (x<=301)): return 'endocrine'
    elif x==302: return 'kidney_transplant'
    elif ((x>=316) & (x<=317)): return 'renal_failure_dialysis'
    elif ((x>=174) & (x<=178)): return 'gi_hemorrhage_ulcer'
    elif ((x>=488) & (x<=490)): return 'hiv'
    elif ((x>=240) & (x<=241)): return 'connective_tissue'
    elif x==397: return 'coagulation'
    elif x==288: return 'obesity_procedure'
    elif ((x>=396) & (x<=298)): return 'nutrition_metabolic'
    elif ((x>=433) & (x<=437)): return 'alcohol_drug'
    elif x==430: return 'psychoses'
    elif x==426: return 'depression'
    else: return 'unknown'

In [ ]:
drg_final['drg_code_converted']=drg_final.DRG_CODE.map(lambda x: convert_drg(x))

In [ ]:
drg_final.shape

In [ ]:
icds.shape

In [ ]:
elix_df=pd.merge(drg_final[['SUBJECT_ID', 'HADM_ID', 'drg_code_converted']], 
                 icds[['SUBJECT_ID', 'HADM_ID', 'icd9_converted', 'elixhauser_category']], 
                on=['SUBJECT_ID', 'HADM_ID'], how='right')

In [ ]:
elix_df

In [ ]:
def elix_pmh(x): 
    if ((x['drg_code_converted']=='cardiac') & (x['elixhauser_category']== 'congestive_heart_failure')): return False 
    elif ((x['drg_code_converted']=='cardiac') & (x['elixhauser_category']== 'cardiac_arrhythmias')): return False 
    elif ((x['drg_code_converted']=='cardiac') & (x['elixhauser_category']== 'valvular_disease')): return False 
    elif ((x['drg_code_converted']=='cardiac') & (x['drg_code_converted']=='copd') & (x['elixhauser_category']== 'pulmonary_circulation')): return False 
    elif ((x['drg_code_converted']=='peripheral_vascular') & (x['elixhauser_category']== 'peripheral_vascular')): return False 
    elif ((x['drg_code_converted']=='hypertension') & (x['drg_code_converted']=='cardiac') & (x['drg_code_converted']=='renal')& (x['elixhauser_category']== 'hypertension')): return False 
    elif ((x['drg_code_converted']=='cerebrovascular') & (x['elixhauser_category']== 'paralysis')): return False
    elif ((x['drg_code_converted']=='nervous_system') & (x['elixhauser_category']== 'other_neurological')): return False
    elif ((x['drg_code_converted']=='copd') & (x['drg_code_converted']=='asthma') & (x['elixhauser_category']== 'chronic_pulmonary')): return False
    elif ((x['drg_code_converted']=='diabetes') & (x['elixhauser_category']== 'diabetes_uncomplicated')): return False
    elif ((x['drg_code_converted']=='diabetes') & (x['elixhauser_category']== 'diabetes_complicated')): return False
    elif ((x['drg_code_converted']=='thyroid') & (x['drg_code_converted']=='endocrine') & (x['elixhauser_category']== 'hypothyroidism')): return False
    elif ((x['drg_code_converted']=='kidney_transplant') & (x['drg_code_converted']=='renal_failure_dialysis') & (x['elixhauser_category']== 'renal_failure')): return False
    elif ((x['drg_code_converted']=='liver') & (x['elixhauser_category']== 'liver_disease')): return False
    elif ((x['drg_code_converted']=='hemorrhage_ulcer') & (x['elixhauser_category']== 'peptic_ulcer')): return False
    elif ((x['drg_code_converted']=='leukemia_lymphoma') & (x['elixhauser_category']== 'lymphoma')): return False
    elif ((x['drg_code_converted']=='cancer') & (x['elixhauser_category']== 'metastatic_cancer')): return False
    elif ((x['drg_code_converted']=='connective_tissue') & (x['elixhauser_category']== 'rheumatoid_arthritis')): return False
    elif ((x['drg_code_converted']=='coagulation') & (x['elixhauser_category']== 'coagulopathy')): return False
    elif ((x['drg_code_converted']=='obesity_procedure') & (x['drg_code_converted']=='nutrition_metabolic') & (x['elixhauser_category']== 'obesity')): return False
    elif ((x['drg_code_converted']=='nutrition_metabolic') & (x['elixhauser_category']== 'weight_loss')): return False
    elif ((x['drg_code_converted']=='nutrition_metabolic') & (x['elixhauser_category']== 'fluid_electrolyte')): return False
    elif ((x['drg_code_converted']=='anemia') & (x['elixhauser_category']== 'blood_loss_anemia')): return False
    elif ((x['drg_code_converted']=='anemia') & (x['elixhauser_category']== 'deficiency_anemia')): return False
    elif ((x['drg_code_converted']=='alcohol_drug') & (x['elixhauser_category']== 'alcohol_abuse')): return False
    elif ((x['drg_code_converted']=='alcohol_drug') & (x['elixhauser_category']== 'drug_abuse')): return False
    elif ((x['drg_code_converted']=='psychoses') & (x['elixhauser_category']== 'psychoses')): return False
    elif ((x['drg_code_converted']=='depression') & (x['elixhauser_category']== 'depression')): return False
    else: return True

In [ ]:
elix_df['true_comorbidity']=elix_df.apply(elix_pmh, axis=1)

In [ ]:
elix_df=elix_df[elix_df.true_comorbidity==True]
elix_final=pd.DataFrame(elix_df.groupby(['SUBJECT_ID', 'HADM_ID'])['elixhauser_category'].apply(','.join))

In [ ]:
elix_final.reset_index()
elix_final['prior_comorbidities']=elix_final.elixhauser_category.map(lambda x: ', '.join(set(x.split(','))))
elix_final.pop('elixhauser_category')
print('')

In [ ]:
elix_final=elix_final.reset_index()

In [ ]:
len(elix_final.HADM_ID.unique())

In [ ]:
elix_final.to_csv('data/MIMIC_001_Elixhauser.csv', index=False)